In [1]:
# Basic setup for displaying bokeh plots in jupyter.
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
# Additional requirements for Stacked Bar plot.
from bokeh.core.properties import value
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [2]:
# NumPy imports.
import numpy as np
import cPickle as pickle
import os
cwd = os.getcwd()
print(cwd)

/Users/kevin/dev/nml/paper_platform/scripts


In [3]:
# Raw data.
raws = pickle.load(open('2018-04-03-14-01-41-627705_singlerun_100.p', 'rb'))
print(type(raws))
print(len(raws))
print(raws[0])

<type 'list'>
100
{'/media/kevin/d640/enterobase_db/ESC_BA1121AA_AS.fasta': [{u'job_ptstx1_datastruct': [u'Tue, 03 Apr 2018 18:02:37 GMT', u'Tue, 03 Apr 2018 18:02:38 GMT', 1.0]}, {u'job_amr_dict': [u'Tue, 03 Apr 2018 18:03:30 GMT', u'Tue, 03 Apr 2018 18:03:30 GMT', 0.0]}, {u'job_ectyper_beautify_serotype': [u'Tue, 03 Apr 2018 18:01:57 GMT', u'Tue, 03 Apr 2018 18:01:58 GMT', 1.0]}, {u'job_ectyper_beautify_vf': [u'Tue, 03 Apr 2018 18:01:55 GMT', u'Tue, 03 Apr 2018 18:01:55 GMT', 0.0]}, {u'job_ectyper_datastruct_vf': [u'Tue, 03 Apr 2018 18:01:55 GMT', u'Tue, 03 Apr 2018 18:01:56 GMT', 1.0]}, {u'job_pteae': [u'Tue, 03 Apr 2018 18:01:56 GMT', u'Tue, 03 Apr 2018 18:03:28 GMT', 92.0]}, {u'job_ptstx1_dict': [u'Tue, 03 Apr 2018 18:02:37 GMT', u'Tue, 03 Apr 2018 18:02:37 GMT', 0.0]}, {u'job_ectyper_serotype': [u'Tue, 03 Apr 2018 18:01:52 GMT', u'Tue, 03 Apr 2018 18:01:57 GMT', 5.0]}, {u'job_pteae_dict': [u'Tue, 03 Apr 2018 18:03:28 GMT', u'Tue, 03 Apr 2018 18:03:28 GMT', 0.0]}, {u'job_ptstx1_be

In [4]:
# Tabulate runtimes per module.
parsed = {
    'qc':[],
    'databaseid':[],
    'serotype':[],
    'vf':[],
    'amr':[],
    'stx1':[],
    'stx2':[],
    'eae':[],
    'total': []
}
for outerd in raws:
    qc=0
    databaseid=0
    serotype=0
    vf=0
    amr=0
    stx1=0
    stx2=0
    eae=0
    total=0
    for d in outerd.values()[0]:
        if 'stx1' in d.keys()[0]:
            stx1 += d.values()[0][2]
        elif 'stx2' in d.keys()[0]:
            stx2 += d.values()[0][2]
        elif 'eae' in d.keys()[0]:
            eae += d.values()[0][2]
        elif 'vf' in d.keys()[0]:
            vf += d.values()[0][2]
        elif 'amr' in d.keys()[0]:
            amr += d.values()[0][2]
        elif 'serotype'in d.keys()[0]:
            serotype += d.values()[0][2]
        elif 'job_id' in d.keys()[0]:
            databaseid += d.values()[0][2]
        elif 'turtle' in d.keys()[0]:
            databaseid += d.values()[0][2]
        elif 'job_qc' in d.keys()[0]:
            qc += d.values()[0][2]
        elif 'total' in d.keys()[0]:
            total += d.values()[0][2]
    parsed['qc'].append(qc)
    parsed['databaseid'].append(databaseid)
    parsed['serotype'].append(serotype)
    parsed['vf'].append(vf)
    parsed['amr'].append(amr)
    parsed['stx1'].append(stx1)
    parsed['stx2'].append(stx2)
    parsed['eae'].append(eae)
    parsed['total'].append(total)

In [5]:
import numpy as np
from bokeh.models import ColumnDataSource
from bokeh.models.glyphs import Line

colormap = {'qc':'red',
            'databaseid':'green',
            'serotype':'blueviolet',
            'vf':'crimson',
            'amr':'firebrick',
            'stx1':'darksalmon',
            'stx2':'darkorange',
            'eae':'darkgoldenrod',
            'total': 'blue'}
colors = [colormap[x] for x in parsed.keys() for i in range(len(parsed.values()[0]))]

x_runs = [i for l in parsed.values() for i,x in enumerate(l)]
y_times = [x for l in parsed.values() for x in l]

p = figure(title = "Timings for Individual Runs",width=500,
           height=500,)
p.xaxis.axis_label = 'Run #'
p.yaxis.axis_label = 'Runtime (seconds)'

print(np.mean(parsed['total']))

lines_source = ColumnDataSource(data=dict(y=[np.mean(parsed['total']) for i in range(len(parsed['total']))], x=[i for i,x in enumerate(parsed['total'])]))
line = Line(x='x',y='y', line_color="#666699", line_width=2)

p.add_glyph(lines_source, line)

p.circle(x_runs, y_times,
         color=colors, fill_alpha=0.2, size=10)

show(p)

129.53


In [6]:
avgs = {k:np.mean(parsed[k]) for k in parsed}
print(avgs)

{'databaseid': 35.38, 'amr': 93.78, 'vf': 4.24, 'stx1': 4.26, 'stx2': 2.88, 'eae': 28.83, 'serotype': 6.24, 'qc': 0.69, 'total': 129.53}


In [7]:
# Plot a histogram of the targets.
x = np.array(parsed['amr'])
print(x.mean())
hist, edges = np.histogram(x, density=True, bins=50)

p1 = figure(title="Histogram of Total Runtimes per Genome",tools="save",)

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="blue", line_color="#033649", alpha=0.5)

show(p1)

93.78


In [8]:
la = [i for i in parsed['total'] if i<140]
lb = [i for i in parsed['total'] if i>=140]
print(np.mean(la), np.mean(lb))

(95.9322033898305, 177.8780487804878)


In [9]:
# Raw data.
raws_batch = pickle.load(open('2018-04-11-10-26-48-900902_spfy_class_11.p', 'rb'))
data_batch = raws_batch.data
type(data_batch)

list

In [10]:
data_batch[1]

[{u'/datastore/2018-04-11-14-30-16-888218/2018-04-11-14-30-16-888218-ESC_CA6411AA_AS.fasta|job_turtle': [u'Wed, 11 Apr 2018 14:30:17 GMT',
   u'Wed, 11 Apr 2018 14:30:18 GMT',
   1.0]},
 {u'/datastore/2018-04-11-14-30-16-888218/2018-04-11-14-30-16-888218-ESC_CA6411AA_AS.fasta|job_amr_dict': [u'Wed, 11 Apr 2018 14:32:09 GMT',
   u'Wed, 11 Apr 2018 14:32:09 GMT',
   0.0]},
 {u'/datastore/2018-04-11-14-30-16-888218/2018-04-11-14-30-16-888218-ESC_CA6411AA_AS.fasta|job_id': [u'Wed, 11 Apr 2018 14:30:17 GMT',
   u'Wed, 11 Apr 2018 14:30:18 GMT',
   1.0]},
 {u'/datastore/2018-04-11-14-30-16-888218/2018-04-11-14-30-16-888218-ESC_CA6411AA_AS.fasta|job_ectyper_vf': [u'Wed, 11 Apr 2018 14:30:18 GMT',
   u'Wed, 11 Apr 2018 14:30:21 GMT',
   3.0]},
 {u'/datastore/2018-04-11-14-30-16-888218/2018-04-11-14-30-16-888218-ESC_CA6411AA_AS.fasta|job_amr_beautify': [u'Wed, 11 Apr 2018 14:32:09 GMT',
   u'Wed, 11 Apr 2018 14:32:10 GMT',
   1.0]},
 {u'/datastore/2018-04-11-14-30-16-888218/2018-04-11-14-30-16-

In [11]:
import pandas as pd
def group_analyses(raws):
    '''Tabulate runtimes per module.
    '''
    assert isinstance(raws, list)
    # "grouped" is on a per job basis.
    grouped = {}
    for outerd in raws:
        assert len(outerd.keys()) == 1
        key = outerd.keys()[0]
        lastkey = key
        if key == 'total':
            analysis = key
        else:
            filename, analysis = key.split('|')
            
        if analysis not in grouped:
            grouped.update({analysis:[]})
        
        grouped[analysis].append(
            outerd.values()[0][2]
        )
    total = grouped.pop('total')
    df = pd.DataFrame(data=grouped)
    return total,df

In [12]:
# Calculate averages.
batches = raws_batch.list_sizes # list_sizes
plot_data = {
    'Batches': [str(b) for b in batches],
    'QC': [],
    'ID': [],
    'VF': [],
    'Serotype': [],
    'AMR': []
}
avt = []
i = 0
for l in data_batch:
    r = group_analyses(l)
    # Remove the total since it's only 1.
    total = r[0]
    df = r[1]
    amr = df[['job_amr','job_amr_beautify','job_amr_datastruct','job_amr_dict']].sum(axis=1)
    serotype = df[['job_ectyper_beautify_serotype','job_ectyper_datastruct_serotype','job_ectyper_serotype']].sum(axis=1)
    vf = df[['job_ectyper_beautify_vf','job_ectyper_datastruct_vf','job_ectyper_vf']].sum(axis=1)
    dbid = df[['job_id','job_turtle']].sum(axis=1)
    qc = df[['job_qc']].sum(axis=1)
    plot_data['QC'].append(qc.mean())
    plot_data['ID'].append(dbid.mean())
    plot_data['VF'].append(vf.mean())
    plot_data['Serotype'].append(serotype.mean())
    plot_data['AMR'].append(amr.mean())
    avt.append(total[0]/batches[i])
    i += 1
print(plot_data)
print(avt)

{'Batches': ['1', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100'], 'VF': [5.0, 4.5, 6.1, 5.833333333333333, 5.85, 6.18, 6.316666666666666, 6.042857142857143, 6.5, 6.088888888888889, 6.39], 'Serotype': [8.0, 6.8, 9.25, 9.833333333333334, 11.375, 12.94, 12.233333333333333, 12.914285714285715, 13.1875, 12.577777777777778, 12.69], 'QC': [6.0, 0.2, 1.1, 1.0333333333333334, 1.1, 1.08, 1.15, 1.1428571428571428, 1.2875, 1.2666666666666666, 1.17], 'AMR': [94.0, 108.5, 108.55, 104.23333333333333, 107.875, 110.34, 104.85, 108.61428571428571, 109.8375, 108.63333333333334, 107.95], 'ID': [4.0, 2.7, 3.1, 2.3333333333333335, 2.5, 2.34, 2.4166666666666665, 2.2, 2.475, 1.9111111111111112, 2.23]}
[104.0, 54.3, 54.5, 52.4, 53.825, 54.8, 52.3, 54.02857142857143, 54.575, 54.044444444444444, 53.44]


In [13]:
# Data.
tasks = ["QC", "ID", "VF", "Serotype", "AMR"] # subtasks
colors = [colormap['qc'], colormap['databaseid'], colormap['vf'],colormap['serotype'], colormap['amr']]

source = ColumnDataSource(data=plot_data)

In [14]:
# Plot.
p = figure(x_range=[str(b) for b in batches],
           plot_height=350,
           title="Runtimes for Analysis Modules",
           x_axis_label="#Total Batch Size",
           y_axis_label="Average Runtime per Genome (seconds)")

p.vbar_stack(tasks, x='Batches', width=0.9, color=colors, source=source,
             legend=[value(x) for x in tasks], name=tasks, alpha=0.5)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "center_right"
p.legend.orientation = "vertical"

p.line(x=[str(b) for b in batches], y=avt, color="cyan", line_width=2)

show(p)

In [15]:
# Calculate averages.
batches = raws_batch.list_sizes # list_sizes
pdata = {
    'Batches': [str(b) for b in batches],
    'QC': [],
    'ID': [],
    'VF': [],
    'Serotype': [],
    'AMR': []
}
i = 0
for l in data_batch:
    r = group_analyses(l)
    # Remove the total since it's only 1.
    total = r[0]
    df = r[1]
    # Total amount of time spent per task (column) per file (row).
    amr = df[['job_amr','job_amr_beautify','job_amr_datastruct','job_amr_dict']].sum(axis=1)
    serotype = df[['job_ectyper_beautify_serotype','job_ectyper_datastruct_serotype','job_ectyper_serotype']].sum(axis=1)
    vf = df[['job_ectyper_beautify_vf','job_ectyper_datastruct_vf','job_ectyper_vf']].sum(axis=1)
    dbid = df[['job_id','job_turtle']].sum(axis=1)
    qc = df[['job_qc']].sum(axis=1)
    # Sum the columns and give me an average over the total number of files in a batch.
    samr = amr.sum(axis=0)
    print(samr/batches[i])
    pdata['QC'].append(qc.mean())
    pdata['ID'].append(dbid.mean())
    pdata['VF'].append(vf.mean())
    pdata['Serotype'].append(serotype.mean())
    pdata['AMR'].append(amr.mean())
    print(total)
    i += 1
# print(plot_data)

94.0
[104.0]
108.5
[543.0]
108.55
[1090.0]
104.23333333333333
[1572.0]
107.875
[2153.0]
110.34
[2740.0]
104.85
[3138.0]
108.61428571428571
[3782.0]
109.8375
[4366.0]
108.63333333333334
[4864.0]
107.95
[5344.0]


In [16]:
l = [i for i in avt[1:]]
print(np.mean(l))

53.82130158730158
